In [14]:
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import re
import string
import nltk

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from gensim.utils import simple_preprocess
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix



In [15]:
! pip install -q kaggle

In [28]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sagarkumarkarn","key":"dfe95288efb6f7e9ec19ba6397ce6d10"}'}

In [29]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [33]:
!pip install kaggle --upgrade


Requirement already up-to-date: kaggle in /usr/local/lib/python3.7/dist-packages (1.5.12)


In [36]:
!kaggle competitions download -c tweet-sentiment-extraction

  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 77.6MB/s]
  0% 0.00/307k [00:00<?, ?B/s]
100% 307k/307k [00:00<00:00, 98.7MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 35.2MB/s]


In [39]:
! mkdir train

In [47]:
! unzip train.csv.zip -d train

Archive:  train.csv.zip
  inflating: train/train.csv         


In [48]:
ls

'kaggle (1).json'       kaggle.json             test.csv
'kaggle (2) (1).json'   sample_data/            train/
'kaggle (2).json'       sample_submission.csv   train.csv.zip


In [50]:
train_data = pd.read_csv('train/train.csv')


In [54]:
train_data.head(20)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [53]:
len(train_data)

27481

In [56]:
train_data.groupby('sentiment').nunique()

,textID,text,selected_text
sentiment,,,
negative,7781,7781,5861
neutral,11118,11117,11111
positive,8582,8582,5537


In [57]:
train_data.head(20)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [59]:
train_data = train_data[['selected_text','sentiment']]
train_data.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [61]:
train_data["selected_text"].isnull().sum()

1

In [63]:
train_data["selected_text"].fillna("No content", inplace = True)

In [65]:
def clean_data(data):
    
    # We need to remove URL's. We remove by using regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # We need to remove emails. We remove by using regular expression
    data = re.sub('\S*@\S*\s?', '', data)

    #     # We need to remove new line characters. We remove by using regular expression 
    data = re.sub('\s+', ' ', data)

    # Removing single quotes
    data = re.sub("\'", "", data)
        
    return data

In [66]:
temp = []
#Splitting pd.Series to list
list_of_data = train_data['selected_text'].values.tolist()
for i in range(len(list_of_data)):
    temp.append(clean_data(list_of_data[i]))
list(temp[:5])

['I`d have responded, if I were going',
 'Sooo SAD',
 'bullying me',
 'leave me alone',
 'Sons of ****,']

In [70]:
#Breaking down to words
def break_into_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(break_into_words(temp))

print(data_words[:20])


[['have', 'responded', 'if', 'were', 'going'], ['sooo', 'sad'], ['bullying', 'me'], ['leave', 'me', 'alone'], ['sons', 'of'], ['some', 'shameless', 'plugging', 'for', 'the', 'best', 'rangers', 'forum', 'on', 'earth'], ['fun'], ['soooo', 'high'], ['both', 'of', 'you'], ['wow', 'just', 'became', 'cooler'], ['as', 'much', 'as', 'love', 'to', 'be', 'hopeful', 'reckon', 'the', 'chances', 'are', 'minimal', 'never', 'gonna', 'get', 'my', 'cake', 'and', 'stuff'], ['like'], ['dangerously'], ['lost'], ['test', 'test', 'from', 'the', 'lg', 'env'], ['uh', 'oh', 'am', 'sunburned'], ['sigh'], ['sick'], ['onna'], ['hes', 'just', 'not', 'that', 'into', 'you']]


In [72]:
def detokenizer(text):
  return TreebankWordDetokenizer().detokenize(text)

In [73]:
#Detokenizing the data
data = []
for i in range(len(data_words)):
    data.append(detokenizer(data_words[i]))
print(data[:5])

['have responded if were going', 'sooo sad', 'bullying me', 'leave me alone', 'sons of']


In [74]:
data_labels = np.array(train_data['sentiment'])
y_float = []
for i in range(len(data_labels)):
    if data_labels[i] == 'neutral':
        y_float.append(0)
    if data_labels[i] == 'negative':
        y_float.append(1)
    if data_labels[i] == 'positive':
        y_float.append(2)
y_float = np.array(y_float)
data_labels = tf.keras.utils.to_categorical(y_float, 3, dtype="float32")
del y_float

In [75]:
len(data_labels)

27481

In [76]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...   68  146   41]
 [   0    0    0 ...    0  397   65]
 [   0    0    0 ...    0    0   11]
 ...
 [   0    0    0 ...  372   10    3]
 [   0    0    0 ...   24  542    4]
 [   0    0    0 ... 2424  199  657]]


In [77]:
print(data_labels)


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [79]:
X_train, X_test, y_train, y_test = train_test_split(tweets,data_labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

20610 6871 20610 6871


In [82]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(3,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric so that it is not lost on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
645/645 [==============================] - 47s 69ms/step - loss: 0.9059 - accuracy: 0.5945 - val_loss: 0.6562 - val_accuracy: 0.7299

Epoch 00001: val_accuracy improved from -inf to 0.72988, saving model to best_model1.hdf5
Epoch 2/70
645/645 [==============================] - 44s 69ms/step - loss: 0.6042 - accuracy: 0.7616 - val_loss: 0.5278 - val_accuracy: 0.7930

Epoch 00002: val_accuracy improved from 0.72988 to 0.79304, saving model to best_model1.hdf5
Epoch 3/70
645/645 [==============================] - 45s 70ms/step - loss: 0.5170 - accuracy: 0.7959 - val_loss: 0.4931 - val_accuracy: 0.8118

Epoch 00003: val_accuracy improved from 0.79304 to 0.81182, saving model to best_model1.hdf5
Epoch 4/70
645/645 [==============================] - 45s 69ms/step - loss: 0.4808 - accuracy: 0.8124 - val_loss: 0.4861 - val_accuracy: 0.8136

Epoch 00004: val_accuracy improved from 0.81182 to 0.81356, saving model to best_model1.hdf5
Epoch 5/70
645/645 [==============================]

In [84]:
#Let's load the best model obtained during training
best_model = keras.models.load_model("best_model1.hdf5")


In [86]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=2)
print('Model total loss and accuracy: ',test_loss,test_accuracy)


215/215 - 2s - loss: 0.4318 - accuracy: 0.8469
Model total loss and accuracy:  0.4317575991153717 0.8468927145004272


In [87]:
predictions = best_model.predict(X_test)


In [99]:
sentiment = ['Neutral','Negative','Positive']
sequence = tokenizer.texts_to_sequences(['i hate youtube ads, they are annoying '])
test = pad_sequences(sequence, maxlen=max_len)
sentiment[np.around(best_model.predict(test), decimals=0).argmax(axis=1)[0]]

'Negative'